# Shoreline extraction at the benchmark sites with CoastSat


This notebook shows how to extract time-series of shoreline change at the 4 benchmark sites using the CoastSat toolbox.

## Initial settings

Refer to the **Installation** section of the [CoastSat README](https://github.com/kvos/CoastSat) for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. If that step has been completed correctly, the following packages should be imported without any problem.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.ion()
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from scipy import interpolate
from scipy import stats
import pytz
import json 
# load coastsat package located under algorithms/COASTSAT/
from coastsat import SDS_download, SDS_preprocess, SDS_shoreline, SDS_tools, SDS_transects

# filepaths to all the datasets
fp_datasets = os.path.join(os.path.join(os.pardir,os.pardir,'datasets'))
names_datasets = os.listdir(fp_datasets)
names_datasets = [_ for _ in names_datasets if _ not in ['README.md','sites_info.txt']]
print('\nBenchmark datasets available:\n%s'%(names_datasets))
                
# load site info dict if exists or create
fp_info = os.path.join(fp_datasets,'sites_info.txt')
if os.path.exists(fp_info):
    with open(fp_info,'r') as f: sites_info = json.load(f)  
    print('\nLoaded sites_info.txt.')
else:
    sites_info = {'NARRABEEN':{'beach_slope':0.1,'contour_level':0.585,'epsg':28356},
                  'DUCK':{'beach_slope':0.1,'contour_level':0.7,'epsg':32119},
                  'TRUCVERT':{'beach_slope':0.05,'contour_level':1.5,'epsg':32630},
                  'TORREYPINES':{'beach_slope':0.045,'contour_level':0.792,'epsg':26946},
                 }
    with open(fp_info,'w') as f: json.dump(sites_info,f,indent=4)
for key in sites_info.keys(): print('%s: %s'%(key,sites_info[key]))


Benchmark datasets available:
['DUCK', 'NARRABEEN', 'TORREYPINES', 'TRUCVERT']

Loaded sites_info.txt.
NARRABEEN: {'beach_slope': 0.1, 'contour_level': 0.585, 'epsg': 28356}
DUCK: {'beach_slope': 0.1, 'contour_level': 0.7, 'epsg': 32119}
TRUCVERT: {'beach_slope': 0.05, 'contour_level': 1.5, 'epsg': 32630}
TORREYPINES: {'beach_slope': 0.045, 'contour_level': 0.792, 'epsg': 26946}


## 1. Retrieval of the images from GEE for the test sites

Here we download all the Landsat (5,7,8,9) amd Sentinel-2 images between 1984 and 2022 for the benchmark sites using the provided ROIs.

In [2]:
# satellites to include
sat_list = ['L5','L7','L8','S2']
# choose Landsat collection 'C01' or 'C02'
landsat_collection = 'C01'
# dates
dates = ['1984-01-01', '2022-01-01']
# folder to save the images in
fp_images = os.path.join(os.getcwd(),'satellite_data')
# loop over each site
for sitename in names_datasets:
    print(sitename)
    data_folder = os.path.join(fp_datasets,sitename)
    # load polygon ROI
    fn_polygon = os.path.join(data_folder, '%s_polygon.geojson'%sitename)
    gdf_polygon = gpd.read_file(fn_polygon)
    polygon = np.array(gdf_polygon.loc[0,'geometry'].exterior.coords)
    # create inputs dictionary
    inputs = {}
    inputs['sitename'] = sitename
    inputs['polygon'] = [[[_[0], _[1]] for _ in polygon]]
    inputs['sat_list'] = sat_list
    inputs['dates'] = dates
    inputs['landsat_collection'] = landsat_collection
    inputs['filepath'] = fp_images
    if os.path.exists(os.path.join(fp_images,sitename,sitename+'_metadata.pkl')): continue
    # download the imagery
    metadata = SDS_download.retrieve_images(inputs)

DUCK
NARRABEEN
TORREYPINES
TRUCVERT


## 2. Extraction of shoreline time-series along the transects

In this cell we map the position of the shoreline on the satellite images, compute the intersections between the shorelines and the transects and perform a tidal correction.

In [ ]:
# extract shoreline time-series at each site
inputs = {}
inputs['filepath'] =  os.path.join(os.getcwd(),'satellite_data')
inputs['landsat_collection'] = 'C01'
inputs['dates'] =  ['1984-01-01', '2022-01-01']
inputs['sat_list'] = ['L5','L7','L8']
for sitename in names_datasets:
    print('Processing %s'%sitename)
    inputs['sitename'] = sitename
    #########################################################################################################
    # Map shorelines
    #########################################################################################################
    # load metadata
    metadata = SDS_download.get_metadata(inputs)
    # remove S2 in not in the inputs
    if 'S2' not in inputs['sat_list']:  metadata.pop('S2');
    # load settings file
    fp_settings = 'settings_shoreline_mapping.txt'
    with open(fp_settings,'r') as f: settings = json.load(f)
    settings['inputs'] = inputs
    # take epsg from sites_info 
    settings['output_epsg'] = sites_info[sitename]['epsg']
    # load reference shoreline
    data_folder = os.path.join(fp_datasets,sitename)
    fn_refsl = os.path.join(data_folder, '%s_reference_shoreline.geojson'%sitename)
    gdf_refsl = gpd.read_file(fn_refsl)
    print('Loaded reference shoreline in epsg:%d'%gdf_refsl.crs.to_epsg(), end='...')
    gdf_refsl.to_crs(epsg=settings['output_epsg'], inplace=True)
    print('converted to epsg:%d'%gdf_refsl.crs.to_epsg())
    refsl = np.array(gdf_refsl.loc[0,'geometry'].coords)
    settings['reference_shoreline'] = refsl
    # extract shorelines from all images (creates or loads output.pkl)
    fn_output = os.path.join(inputs['filepath'], sitename, sitename + '_output' + '.pkl')
    if os.path.exists(fn_output):
        with open(fn_output, 'rb') as f:
            output = pickle.load(f)
        print('loaded existing file at %s'%fn_output)
    else:
        %matplotlib qt
        output = SDS_shoreline.extract_shorelines(metadata, settings)
        # remove duplicates (images taken on the same date by the same satellite)
        output = SDS_tools.remove_duplicates(output)
        # remove inaccurate georeferencing (set threshold to 10 m)
        output = SDS_tools.remove_inaccurate_georef(output, 10)
        # remove bad images based on average classification (only works for same resolution, Landsat)
        if not 'S2' in metadata.keys():
            # settings for discarding the badly classified images"
            settings['prc_pixel'] = 0.15    # minimum percentage of change from land to water for defining the wet/dry areas
            settings['prc_image'] = 0.3     # maximum percentage of misclassified pixels allowed in the image, otherwise discarded
            output = SDS_shoreline.remove_bad_images(output,settings)
        else:
            output.pop('im_classif')
            with open(os.path.join(filepath, sitename + '_output' + '.pkl'), 'wb') as f:
                pickle.dump(output,f)
    # load transects
    fn_transects = os.path.join(data_folder, '%s_transects.geojson'%sitename)
    gdf_transects = gpd.read_file(fn_transects)
    print('Loaded transects in epsg:%d'%gdf_transects.crs.to_epsg())
    gdf_transects.to_crs(epsg=settings['output_epsg'], inplace=True)
    print('Converted to epsg:%d'%gdf_transects.crs.to_epsg())
    # put transects into a dictionary with their name
    transects = dict([])
    for i in gdf_transects.index:
        transects[gdf_transects.loc[i,'name']] = np.array(gdf_transects.loc[i,'geometry'].coords)
    %matplotlib inline
    fig = plt.figure(figsize=[15,8], tight_layout=True)
    plt.axis('equal')
    plt.xlabel('Eastings')
    plt.ylabel('Northings')
    plt.grid(linestyle=':', color='0.5')
    for i in range(len(output['shorelines'])):
        sl = output['shorelines'][i]
        date = output['dates'][i]
        plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
    for i,key in enumerate(list(transects.keys())):
        plt.plot(transects[key][0,0],transects[key][0,1], 'bo', ms=5)
        plt.plot(transects[key][:,0],transects[key][:,1],'k-',lw=1)
        plt.text(transects[key][0,0]-100, transects[key][0,1]+100, key,
                    va='center', ha='right', bbox=dict(boxstyle="square", ec='k',fc='w'))
    fig.savefig(os.path.join(inputs['filepath'], inputs['sitename'], 'mapped_shorelines.jpg'),dpi=200)
    #########################################################################################################
    # Compute intersections
    #########################################################################################################
    # load settings file
    fp_settings = 'settings_transect_intersections.txt'
    with open(fp_settings,'r') as f: settings_transects = json.load(f)
    cross_distance = SDS_transects.compute_intersection_QC(output, transects, settings_transects)
    #########################################################################################################
    # Tidal correction
    #########################################################################################################
    # load tide time-series
    fn_tides = os.path.join(data_folder,'%s_tides.csv'%sitename)
    tide_data = pd.read_csv(fn_tides, parse_dates=['dates'])
    dates_ts = [pytz.utc.localize(_.to_pydatetime()) for _ in tide_data['dates']]
    tides_ts = np.array(tide_data['tides'])
    # get tide levels corresponding to the time of image acquisition
    dates_sat = output['dates']
    tides_sat = SDS_tools.get_closest_datapoint(dates_sat, dates_ts, tides_ts)
    # plot the subsampled tide data
    fig, ax = plt.subplots(1,1,figsize=(15,4), tight_layout=True)
    ax.grid(which='major', linestyle=':', color='0.5')
    ax.plot(tide_data['dates'], tide_data['tides'], '-', color='0.6', label='all time-series')
    ax.plot(dates_sat, tides_sat, '-o', color='k', ms=6, mfc='w',lw=1, label='image acquisition')
    ax.set(ylabel='tide level [m]',xlim=[dates_sat[0],dates_sat[-1]], title='Water levels at the time of image acquisition')
    ax.legend()
    fig.savefig(os.path.join(inputs['filepath'], inputs['sitename'], 'tide_timeseries.jpg'),dpi=200) 
    # tidal correction along each transect
    reference_elevation = sites_info[sitename]['contour_level'] # elevation at which you would like the shoreline time-series to be
    beach_slope = sites_info[sitename]['beach_slope']
    cross_distance_tidally_corrected, tidal_corrections = {},{}
    for key in cross_distance.keys():
        correction = (tides_sat-reference_elevation)/beach_slope
        tidal_corrections[key] = correction
        cross_distance_tidally_corrected[key] = cross_distance[key] + correction
    # remove outliers
    print('removing outliers...')
    cross_distance_tidally_corrected = SDS_transects.reject_outliers(cross_distance_tidally_corrected,output,settings_transects)
    # save in .csv files
    fp_raw_timeseries = os.path.join(inputs['filepath'], inputs['sitename'], 'raw_timeseries')
    fp_tc_timeseries = os.path.join(inputs['filepath'], inputs['sitename'], 'tidally_corected_timeseries')
    if not os.path.exists(fp_raw_timeseries): os.makedirs(fp_raw_timeseries)
    if not os.path.exists(fp_tc_timeseries): os.makedirs(fp_tc_timeseries)
    for key in cross_distance_tidally_corrected.keys():
        out_dict = dict([])
        out_dict['dates'] = output['dates']
        out_dict[key] = cross_distance_tidally_corrected[key]
        out_dict['satname'] = output['satname']
        df = pd.DataFrame(out_dict)
        df.index=df['dates']
        df.pop('dates')
        # save tidally_corrected timeseries
        fn = os.path.join(fp_tc_timeseries,'%s_timeseries_tidally_corrected.csv'%key)
        df.to_csv(fn, sep=',')
        # save raw timeseries
        out_dict[key] = cross_distance_tidally_corrected[key] - tidal_corrections[key]
        df = pd.DataFrame(out_dict)
        df.index=df['dates']
        df.pop('dates')
        fn = os.path.join(fp_raw_timeseries,'%s_timeseries_raw.csv'%key)
        df.to_csv(fn, sep=',')
    # plot time-series
    %matplotlib qt
    fp_plots = os.path.join(inputs['filepath'], inputs['sitename'], 'plots')
    if not os.path.exists(fp_plots): os.makedirs(fp_plots)
    month_colors = plt.cm.get_cmap('tab20')
    for key in cross_distance_tidally_corrected.keys():
        chainage = cross_distance_tidally_corrected[key]
        # remove nans
        idx_nan = np.isnan(chainage)
        dates_nonan = [dates_sat[_] for _ in np.where(~idx_nan)[0]]
        chainage = chainage[~idx_nan] 
        # compute shoreline monthly averages
        dict_month, dates_month, chainage_month, list_month = SDS_transects.monthly_average(dates_nonan, chainage)
        # plot monthly averages
        fig,ax=plt.subplots(1,1,figsize=[14,4],tight_layout=True)
        ax.grid(b=True,which='major', linestyle=':', color='0.5')
        ax.set_title('Time-series at %s'%key, x=0, ha='left')
        ax.set(ylabel='distance [m]')
        ax.plot(dates_nonan, chainage,'+', lw=1, color='k', mfc='w', ms=4, alpha=0.5,label='raw datapoints')
        ax.plot(dates_month, chainage_month, '-', lw=2, color='k', mfc='w', ms=4, label='monthly-averaged')
        # for k,month in enumerate(dict_month.keys()):
        #     ax.plot(dict_month[month]['dates'], dict_month[month]['chainages'],
        #              'o', mec='k', color=month_colors(k), label=month,ms=5)
        ax.legend(loc='lower left',ncol=7,markerscale=1.5,frameon=True,edgecolor='k',columnspacing=1)
        fig.savefig(os.path.join(fp_plots,'%s_timeseries.jpg'%key),dpi=200)
        plt.close(fig)


Processing DUCK
Loaded reference shoreline in epsg:4326...converted to epsg:32119
Mapping shorelines:
L5:   100%
L7:   100%
L8:   100%
0 duplicates
10 bad georef
0 / 641 images different size
6 / 641 images wrong classif
0% images removed
Mean clasification image was saved in C:\Users\z5030440\Documents\SDS_Benchmark\algorithms\COASTSAT\satellite_data\DUCK\classif_qa.jpg
Loaded transects in epsg:4326
Converted to epsg:32119
Extracting closest points: 100%removing outliers...
Processing NARRABEEN
Loaded reference shoreline in epsg:4326...converted to epsg:28356
Mapping shorelines:
L5:   100%
L7:   100%
L8:   100%
89 duplicates
0 bad georef
0 / 545 images different size
5 / 545 images wrong classif
0% images removed
Mean clasification image was saved in C:\Users\z5030440\Documents\SDS_Benchmark\algorithms\COASTSAT\satellite_data\NARRABEEN\classif_qa.jpg
Loaded transects in epsg:4326
Converted to epsg:28356
Extracting closest points: 100%removing outliers...
Processing TORREYPINES
Loaded 

## 3. Comparison with groundtruth

In [ ]:
# parameters for comparing and plotting the two time-series
settings = {'min_days':  3,       # numbers of days difference under which to use nearest neighbour interpolation
            'max_days':  10,      # maximum number of days difference to do a comparison
            'binwidth':  3,       # binwidth for histogram plotting
            'lims':      [-50,50] # cross-shore change limits for plotting purposes
           }
inputs = {}
inputs['filepath'] =  os.path.join(os.getcwd(),'satellite_data')
selected_transects = {
    'NARRABEEN': ['PF1','PF2','PF4','PF6','PF8'],
    'DUCK':      ['-91','1','1006','1097'],
    'TRUCVERT':   ['-400','-300','-200','-100'],
    'TORREYPINES':['PF525','PF530','PF535','PF540','PF545','PF570','PF575','PF580','PF585','PF593','PF598'],    
    }
for sitename in names_datasets:
    print('Processing %s'%sitename)
    inputs['sitename'] = sitename
    fp_tc_timeseries = os.path.join(inputs['filepath'], inputs['sitename'], 'tidally_corected_timeseries')
    # load survey data
    data_folder = os.path.join(fp_datasets,sitename)
    with open(os.path.join(data_folder, '%s_groundtruth.pkl'%sitename), 'rb') as f:
        gt = pickle.load(f)
    chain_sat_all = []
    chain_sur_all = []
    satnames_all = []
    fp_comp = os.path.join(inputs['filepath'],inputs['sitename'],'comparison')
    if not os.path.exists(fp_comp): os.makedirs(fp_comp)
    for key in selected_transects[sitename]:
        # load satellite time-series
        fp_ts = os.path.join(fp_tc_timeseries,'%s_timeseries_tidally_corrected.csv'%key)
        ts = pd.read_csv(fp_ts,parse_dates=['dates'])
        ts_sat, ts_sur, satnames, fig = SDS_tools.compare_timeseries(ts,gt,key,settings)
        fig.savefig(os.path.join(fp_comp,'%s_comparison_transect_%s.jpg'%(sitename,key)), dpi=150)
        plt.close(fig)
        chain_sat_all = np.append(chain_sat_all,ts_sat)
        chain_sur_all = np.append(chain_sur_all,ts_sur)
        satnames_all = satnames_all + satnames  
    # calculate statistics for all transects together
    chain_error = chain_sat_all - chain_sur_all
    slope, intercept, rvalue, pvalue, std_err = stats.linregress(chain_sur_all, chain_sat_all)
    R2 = rvalue**2
    rmse = np.sqrt(np.mean((chain_error)**2))
    mean = np.mean(chain_error)
    std = np.std(chain_error)
    q90 = np.percentile(np.abs(chain_error), 90)
    fig,ax = plt.subplots(1,2,figsize=(15,5), tight_layout=True)
    # histogram
    ax[0].grid(which='major',linestyle=':',color='0.5')
    ax[0].axvline(x=0, ls='--', lw=1.5, color='k')
    binwidth = settings['binwidth']
    bins = np.arange(min(chain_error), max(chain_error) + binwidth, binwidth)
    density = ax[0].hist(chain_error, bins=bins, density=True, color='0.6', edgecolor='k', alpha=0.5)
    mu, std = stats.norm.fit(chain_error)
    pval = stats.normaltest(chain_error)[1]
    xlims = settings['lims']
    x = np.linspace(xlims[0], xlims[1], 100)
    p = stats.norm.pdf(x, mu, std)
    ax[0].plot(x, p, 'r-', linewidth=1)
    ax[0].set(xlabel='error [m]', ylabel='pdf', xlim=settings['lims'], title=sitename)
    str_stats = ' rmse = %.1f\n mean = %.1f\n std = %.1f\n q90 = %.1f' % (rmse, mean, std, q90)
    ax[0].text(0, 0.98, str_stats,va='top', transform=ax[0].transAxes,fontsize=14)
    # boxplot
    data = []
    median_data = []
    n_data = []
    ax[1].yaxis.grid()
    for k,sat in enumerate(list(np.unique(satnames_all))):
        idx = np.where([_ == sat for _ in satnames_all])[0]
        data.append(chain_error[idx])
        median_data.append(np.median(chain_error[idx]))
        n_data.append(len(chain_error[idx]))
    bp = ax[1].boxplot(data,0,'k.', labels=list(np.unique(satnames_all)), patch_artist=True)
    for median in bp['medians']:
        median.set(color='k', linewidth=1.5)
    for j,boxes in enumerate(bp['boxes']):
        boxes.set(facecolor='C'+str(j))
        ax[1].text(j+1,median_data[j]+1, '%.1f' % median_data[j], horizontalalignment='center', fontsize=14)
        ax[1].text(j+1+0.35,median_data[j]+1, ('n=%.d' % int(n_data[j])), ha='center', va='center', fontsize=12, rotation='vertical')
    ax[1].set(ylabel='error [m]', ylim=settings['lims']);
    fig.savefig(os.path.join(inputs['filepath'], inputs['sitename'],'%s_comparison_transect_all.jpg'%(sitename)), dpi=150)
    fig.savefig(os.path.join(os.getcwd(),'%s_comparison_transect_all.jpg'%(sitename)), dpi=150)


In [ ]:
%matplotlib inline